In [ ]:
# Install Ollama and Huggingface packages
%pip install llama-index-llms-ollama llama-index-embeddings-huggingface llama-index networkx llama-index-graph-stores-neo4j

### RAG using Vector Embeddings

In [80]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Use local model
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:latest", request_timeout=120)

# Load documents
documents = SimpleDirectoryReader('files/test').load_data()

# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")
# Build the index with local embeddings
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Set up query engine using Ollama
query_engine = index.as_query_engine(llm=llm)

# Query
response = query_engine.query("Who teaches in romania and what courses do they teach?")
print(response)


The teaching faculty at a specific institution in Romania, which I'll refer to as a key location, includes an individual who instructs in semantic web. This instructor has expertise in databases and conception modelling, suggesting a broad range of academic subjects. Teaching in this field can be demanding due to the need for patience, long hours spent with students, and navigating various topics related to data management and system design.


### RAG using KG

In [78]:
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Use local model
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:latest", request_timeout=120)

# Load documents
documents = SimpleDirectoryReader('files/test').load_data()

# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")


graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    # max_triplets_per_chunk=2,
    storage_context=storage_context,
    embed_model='local',
    llm=llm
)

# Set up query engine using Ollama
query_engine = kg_index.as_query_engine(llm=llm)

# Query
response = query_engine.query("Who teaches in romania and what courses do they teach?")
print(response)


Anna is located in Cluj Napoca, which is a city in Romania. She teaches certain subjects that are relevant to her academic background, including semantic web, databases, and conception modelling.


In [42]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.tools import QueryEngineTool

# Load documents
documents = SimpleDirectoryReader('files/test').load_data()


# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Build Knowledge Graph Index
graph_store = SimpleGraphStore()
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    graph_store=graph_store,
    embed_model='local',
    llm=llm,
    max_triplets_per_chunk=10
)

# Create query engines
kg_engine = kg_index.as_query_engine(llm=llm)

# Combine into a Router engine
tools = [
    QueryEngineTool.from_defaults(
        query_engine=kg_engine,
        description="Use for fact-based or entity relationship queries."
    )
]
router_engine = RouterQueryEngine.from_defaults(query_engine_tools=tools, llm=llm)

# Query
response = router_engine.query("What are the courses taught by a romanian professor?")
print(response)


Romanian professors typically teach courses in Romanian language and literature, history, philosophy, and cultural studies. They may also instruct in fields such as economics, politics, or international relations, reflecting the country's rich academic heritage.


VERSION BOTH VECTOR AND KG

In [23]:
# Use local model


from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:latest", request_timeout=500)

In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.tools import QueryEngineTool

# Load documents
documents = SimpleDirectoryReader('files/test').load_data()


# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Build Knowledge Graph Index
graph_store = SimpleGraphStore()
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    graph_store=graph_store,
    embed_model='local',
    llm=llm,
    max_triplets_per_chunk=10
)

# Build the index with local embeddings
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Create query engines
kg_engine = kg_index.as_query_engine(llm=llm)
vector_engine = vector_index.as_query_engine(llm=llm)

# Combine into a Router engine
tools = [
    QueryEngineTool.from_defaults(
        query_engine=kg_engine,
        description="Use for fact-based or entity relationship queries."
    ),
    QueryEngineTool.from_defaults(
        query_engine=vector_engine,
        description="Use for general or contextual document questions."
    )
]
router_engine = RouterQueryEngine.from_defaults(query_engine_tools=tools, llm=llm)

# Query
response = router_engine.query("Summarize each document in a few sentences.")
print(response)


Document 1:
A collaborative research project has been initiated by two prominent researchers, Dr. Grant and Dr. Reed. Dr. Lisa Grant is a molecular biologist, while Dr. Alan Reed is a data scientist from NovaTech, an AI company based in the U.S. The nature of their collaboration involves data analysis.

Document 2:
Dr. Lisa Grant and Dr. Alan Reed have partnered to work on a research project under BioCore's Emerging Innovations Division. Their collaboration combines expertise from molecular biology and data science, reflecting the interdisciplinary approach often seen in AI-driven research projects.


In [27]:
print(query_engine.query("Who is Lisa Grant and what is she collaborating on?"))

Lisa Grant is an individual with expertise in firefighting. She is currently collaborating with Dr. Alan Reed, a data scientist from NovaTech, on a data analysis project.


# Old

In [22]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama

# Step 1: Use Ollama as the LLM
llm = Ollama(model="llama3.2:latest")  # or "mistral", "phi3", etc.

# Step 2: Load documents
documents = SimpleDirectoryReader("files/test").load_data()

# Step 3: Create Knowledge Graph Index with the custom LLM
index = KnowledgeGraphIndex.from_documents(
    documents,
    # embed_model='local',
    max_triplets_per_chunk=2,
    use_async=True,
    llm=llm  # pass the Ollama model to the index
)

# Step 4: Query
query_engine = index.as_query_engine()
response = query_engine.query("Who is Lisa Grant and what is she working on?")

print(response)


ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [64]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load documents
documents = SimpleDirectoryReader('files/test').load_data()

# Set up LLM and embeddings
llm = Ollama(model="llama3.2:latest", timeout=500)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Apply settings (optional global config)
Settings.embed_model = embed_model
Settings.llm = llm

# Create the Knowledge Graph Index
index = KnowledgeGraphIndex.from_documents(documents)

# Set up query engine (uses semantic similarity over graph + LLM reasoning)
query_engine = index.as_query_engine()

# Query the knowledge graph
response = query_engine.query("Summarize each document in a few sentences.")
print(response)


Unfortunately, there are no documents provided for summarization.
